In [1]:
import os
import pandas as pd
import sklearn
import numpy as np
from matplotlib import pyplot as plt
from sklearn.utils import shuffle
from brand_lists import *
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)

#converte i risultati da {-1,0,1} a {0, 0.5, 1}
def convert_value(val):
    if val == 1: return 1
    if val == -1: return 0
    if val == 0: return 0.5

In [3]:
#Caricamento tabella UTENTI-TAGS
users_tags_df = pd.read_excel('sentiment_results/users_tags_table_vader.xlsx', index_col=0)
users_tags_df

,User,baby,car,chocolate,clothing,coffee,computer,fashion,home,household appliances,italian,licorice,office,pasta,scent,sporty,technology
0,007maybe,1,1,0,0,0,0,0,-1,0,0,0,0,0,0,1,0
1,21Phylk,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
2,ABO_mondo,0,1,0,0,0,0,1,1,0,1,0,0,1,0,0,1
3,ACE2276,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,AGnomblea,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,werewolfdonut,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
494,wilhelmdonkor27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
495,wittystwitty,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
496,xkj1985x,0,1,0,0,0,0,0,0,0,-1,0,0,0,0,1,0


In [4]:
#Conversione della scala dei valori da {-1,0,1} a {0, 0.5, 1}
for tag in all_brands_tags:
    users_tags_df[tag] = users_tags_df[tag].apply(lambda x: convert_value(x))

users_tags_df

,User,baby,car,chocolate,clothing,coffee,computer,fashion,home,household appliances,italian,licorice,office,pasta,scent,sporty,technology
0,007maybe,1.0,1.0,0.5,0.5,0.5,0.5,0.5,0.0,0.5,0.5,0.5,0.5,0.5,0.5,1.0,0.5
1,21Phylk,0.5,0.5,0.5,1.0,0.5,0.5,1.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
2,ABO_mondo,0.5,1.0,0.5,0.5,0.5,0.5,1.0,1.0,0.5,1.0,0.5,0.5,1.0,0.5,0.5,1.0
3,ACE2276,0.5,1.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
4,AGnomblea,1.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,werewolfdonut,0.5,0.5,0.5,0.5,0.5,0.5,1.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
494,wilhelmdonkor27,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
495,wittystwitty,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
496,xkj1985x,0.5,1.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.5,0.5,0.5,0.5,1.0,0.5


In [5]:
#Liste di utenti e brands:
all_users_list = list(np.unique(users_tags_df["User"])) #lista di utenti #lista di utenti
all_brands_list = brands_dict.keys()

#Inizializzazione della tabella users-brands: (Valori di default= 0.5 [Neutrali])
users_brands_df = pd.DataFrame(all_users_list, columns=['User'])    #creazione del dataframe
for brand in all_brands_list:
    users_brands_df[brand] = 0.5  


#computazione degli indici di compatibilità degli utenti rispetto ai brand:
#ESEMPIO:
# Mario ha ottenuto polarità -1, 0, 0 per i tre tag di Amarelli. 
# Diventeranno 0, 0.5, 0.5 e l’indice di compatibilità tra Mario e Amarelli avrà valore (0 + 0.5 + 0.5) / 3 = 0.33.

#per tutti gli utenti
for username in all_users_list:
    #per tutti i brands
    for brand in all_brands_list:
        #lista dei tags associati al brand
        brand_tags = brands_dict[brand]
        total = 0
        count_tag = 0
        for tag in brand_tags:
            #copia valore di sentiment sulla tabella users_tags:
            sentiment_value = users_tags_df.loc[users_tags_df["User"] == username , tag].values[0]
            total = total + sentiment_value
            count_tag = count_tag +1
        #Il sentiment dell'utente rispetto il brand sarà la media dei sentiment diviso il numero di tags (3)
        users_brands_df.loc[ users_brands_df["User"] == username, brand] = total/count_tag



#Funzione che salva in formato CSV un dataframe
if not os.path.exists('final_results'):
    os.makedirs('final_results')
users_tags_df.to_csv("final_results/users_tags_values.csv", index=False) #salvare il dataframe in un CSV
users_brands_df.to_csv("final_results/users_brands_values.csv", index=False) #salvare il dataframe in un CSV

In [6]:
users_brands_df = pd.read_csv('final_results/users_brands_values.csv')
users_brands_df

,User,alfaromeo,samsung,lavazza,armani,adidas,microsoft,barilla,ferrero,delonghi,amarelli
0,007maybe,0.833333,0.500000,0.500000,0.500000,0.666667,0.500000,0.333333,0.666667,0.333333,0.666667
1,21Phylk,0.500000,0.500000,0.500000,0.833333,0.833333,0.500000,0.500000,0.500000,0.500000,0.500000
2,ABO_mondo,0.833333,0.666667,0.666667,0.833333,0.666667,0.666667,1.000000,0.666667,0.666667,0.500000
3,ACE2276,0.666667,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
4,AGnomblea,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.666667,0.500000,0.666667
...,...,...,...,...,...,...,...,...,...,...,...
493,werewolfdonut,0.500000,0.500000,0.500000,0.666667,0.666667,0.500000,0.500000,0.500000,0.500000,0.500000
494,wilhelmdonkor27,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
495,wittystwitty,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
496,xkj1985x,0.666667,0.500000,0.333333,0.333333,0.666667,0.500000,0.333333,0.333333,0.500000,0.500000
